In [41]:
# imports 
import pandas as pd
import sqlite3
import random
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.io import output_notebook
from datetime import datetime

In [19]:


# Cargar el CSV de clientes
customers_df = pd.read_csv("data/customers-10000.csv")
customers_df.head()


,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,EB54EF1154C3A78,Heather,Callahan,Mosley-David,Lake Jeffborough,Norway,043-797-5229,915.112.1727,urangel@espinoza-francis.net,2020-08-26,http://www.escobar.org/
1,2,10dAcafEBbA5FcA,Kristina,Ferrell,"Horn, Shepard and Watson",Aaronville,Andorra,932-062-1802,(209)172-7124x3651,xreese@hall-donovan.com,2020-04-27,https://tyler-pugh.info/
2,3,67DAB15Ebe4BE4a,Briana,Andersen,Irwin-Oneal,East Jordan,Nepal,8352752061,(567)135-1918,haleybraun@blevins-sexton.com,2022-03-22,https://www.mack-bell.net/
3,4,6d350C5E5eDB4EE,Patty,Ponce,Richardson Group,East Kristintown,Northern Mariana Islands,302.398.3833,196-189-7767x770,hohailey@anthony.com,2020-07-02,https://delacruz-freeman.org/
4,5,5820deAdCF23EFe,Kathleen,Mccormick,Carson-Burch,Andresmouth,Macao,001-184-153-9683x1497,552.051.2979x342,alvaradojesse@rangel-shields.com,2021-01-17,https://welch.info/


In [25]:
#Carga de la base de datos
conn = sqlite3.connect("base/bank.db") 
bt_df = pd.read_csv("base/10000 BT Records.csv") 

bt_df.to_sql("bt_records", conn, if_exists="replace", index=False)
conn.commit()


In [26]:
# Agregar la variable customers id del csv en la db para poder tener una relacion entre ambos datasets
customer_ids = customers_df['Customer Id'].tolist()

# Asignar aleatoriamente un Customer Id a cada fila de bt_df
bt_df['Customer Id'] = [random.choice(customer_ids) for _ in range(len(bt_df))]

# Opcional: verificar las primeras filas con la nueva columna
print(bt_df.head())

# Guardar la tabla actualizada en la base de datos (sobrescribir)
bt_df.to_sql("bt_records", conn, if_exists="replace", index=False)



          Date Description   Deposits Withdrawls     Balance      Customer Id
0  20-Aug-2020        Cash  97,540.00      00.00  170,542.01  Fbd8FAaD2ccC03F
1  20-Aug-2020  Commission      00.00  34,108.40  136,433.61  ad72E2c53510Aed
2  20-Aug-2020         ATM      41.00      00.00  136,474.61  59aa3f2dabdf86f
3  20-Aug-2020    Interest      50.00      00.00  136,524.61  b1BabAFCce090F5
4  20-Aug-2020  Commission      00.00  34,131.15  102,393.46  bE13C300faDAE19


10000

In [27]:
merged_df = pd.merge(bt_df, customers_df, on='Customer Id', how='left')

print(merged_df.head())

merged_df.to_csv("data/merged_dataset.csv", index=False)

          Date Description   Deposits Withdrawls     Balance      Customer Id  \
0  20-Aug-2020        Cash  97,540.00      00.00  170,542.01  Fbd8FAaD2ccC03F   
1  20-Aug-2020  Commission      00.00  34,108.40  136,433.61  ad72E2c53510Aed   
2  20-Aug-2020         ATM      41.00      00.00  136,474.61  59aa3f2dabdf86f   
3  20-Aug-2020    Interest      50.00      00.00  136,524.61  b1BabAFCce090F5   
4  20-Aug-2020  Commission      00.00  34,131.15  102,393.46  bE13C300faDAE19   

   Index First Name Last Name                     Company            City  \
0    677     Dillon      Soto  Avery, Torres and Arellano       Davidberg   
1   8171    Douglas   Bennett              Hatfield Group      Devonmouth   
2   9230   Virginia      Haas                 Fleming Ltd     Rebekahport   
3   9791  Alejandro   Cameron              Richardson Ltd      Louisburgh   
4   7864   Jonathan    Atkins                Zuniga-Lynch  New Jeromeport   

         Country             Phone 1              

# GRAFICAS INTERACTIVAS CON BOKEH

### Grafica por mes para depósitos y retiros

In [44]:

merged_df['Date'] = pd.to_datetime(merged_df['Date'], format='%d-%b-%Y', errors='coerce')

# 3. Corregir columnas numéricas (quitar comas, convertir a float)
merged_df['Deposits'] = merged_df['Deposits'].replace('[\$,]', '', regex=True).astype(float)
merged_df['Withdrawls'] = merged_df['Withdrawls'].replace('[\$,]', '', regex=True).astype(float)

# 4. Crear columna 'Mes' en formato año-mes
merged_df['Mes'] = merged_df['Date'].dt.strftime('%Y-%m')

# 5. Agrupar por mes y sumar depósitos y retiros
resumen = merged_df.groupby('Mes').agg({'Deposits': 'sum', 'Withdrawls': 'sum'}).reset_index()

# 6. Preparar datos para la gráfica
x = []
top = []
colores = []

for _, fila in resumen.iterrows():
    mes = fila['Mes']
    x.append((mes, 'Depósitos'))
    top.append(fila['Deposits'])
    colores.append('green')
    x.append((mes, 'Retiros'))
    top.append(fila['Withdrawls'])
    colores.append('red')

# 7. Crear data source para Bokeh
source = ColumnDataSource(data=dict(x=x, top=top, color=colores))

# 8. Graficar con Bokeh
p = figure(x_range=FactorRange(*x), 
           title="Depósitos y Retiros Totales por Mes",
           x_axis_label='Mes y Tipo', y_axis_label='Monto (USD)',
           height=400, width=900, toolbar_location=None)

p.vbar(x='x', top='top', width=0.8, source=source, fill_color='color')
p.xaxis.major_label_orientation = 1.2

show(p)

<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
C:\Users\adpar\AppData\Local\Temp\ipykernel_16896\3565448367.py:4: SyntaxWarning: invalid escape sequence '\$'
  merged_df['Deposits'] = merged_df['Deposits'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\adpar\AppData\Local\Temp\ipykernel_16896\3565448367.py:5: SyntaxWarning: invalid escape sequence '\$'
  merged_df['Withdrawls'] = merged_df['Withdrawls'].replace('[\$,]', '', regex=True).astype(float)


### Top 10 clientes con más depósitos o retiros acumulados

In [45]:
top_clientes = merged_df.groupby(['Customer Id', 'First Name', 'Last Name']).agg({
    'Deposits': 'sum',
    'Withdrawls': 'sum'
}).reset_index()

top_clientes['Total Movimientos'] = top_clientes['Deposits'] + top_clientes['Withdrawls']

top10 = top_clientes.sort_values(by='Deposits', ascending=False).head(10)

top10['Cliente'] = top10['First Name'] + ' ' + top10['Last Name']

x = []
top = []
colores = []

for _, fila in top10.iterrows():
    cliente = fila['Cliente']
    x.append((cliente, 'Depósitos'))
    top.append(fila['Deposits'])
    colores.append('green')
    x.append((cliente, 'Retiros'))
    top.append(fila['Withdrawls'])
    colores.append('red')

source = ColumnDataSource(data=dict(x=x, top=top, color=colores))

p1 = figure(x_range=FactorRange(*x), 
           title="Top 10 Clientes con más Depósitos y Retiros",
           x_axis_label='Cliente', y_axis_label='Monto (USD)',
           height=400, width=1000, toolbar_location=None)

p1.vbar(x='x', top='top', width=0.8, source=source, fill_color='color')
p1.xaxis.major_label_orientation = 1.2

show(p1)


### Distribución de Depósitos y Retiros por País

In [46]:
por_pais = merged_df.groupby('Country').agg({
    'Deposits': 'sum',
    'Withdrawls': 'sum'
}).reset_index()

por_pais['Total'] = por_pais['Deposits'] + por_pais['Withdrawls']
por_pais = por_pais.sort_values(by='Total', ascending=False).head(10)  # Top 10 países

x = []
top = []
colores = []

for _, fila in por_pais.iterrows():
    pais = fila['Country']
    x.append((pais, 'Depósitos'))
    top.append(fila['Deposits'])
    colores.append('green')
    x.append((pais, 'Retiros'))
    top.append(fila['Withdrawls'])
    colores.append('red')

source = ColumnDataSource(data=dict(x=x, top=top, color=colores))

p2 = figure(x_range=FactorRange(*x), 
           title="Distribución de Depósitos y Retiros por País (Top 10)",
           x_axis_label='País', y_axis_label='Monto (USD)',
           height=400, width=1000, toolbar_location=None)

p2.vbar(x='x', top='top', width=0.8, source=source, fill_color='color')
p2.xaxis.major_label_orientation = 1.2

show(p2)


### Evolución del saldo promedio mensual

In [47]:
merged_df['Balance'] = merged_df['Balance'].replace('[\$,]', '', regex=True).astype(float)

saldo_mensual = merged_df.groupby('Mes')['Balance'].mean().reset_index()

p3 = figure(x_range=saldo_mensual['Mes'], 
            title="Saldo Promedio por Mes",
            x_axis_label='Mes', y_axis_label='Saldo Promedio (USD)',
            height=400, width=900, toolbar_location=None)

p3.line(x=saldo_mensual['Mes'], y=saldo_mensual['Balance'], line_width=2, color='blue', legend_label='Saldo Promedio')
p3.circle(x=saldo_mensual['Mes'], y=saldo_mensual['Balance'], size=8, color='blue')

p3.xaxis.major_label_orientation = 1.2
p3.legend.location = "top_left"

show(p3)


<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
C:\Users\adpar\AppData\Local\Temp\ipykernel_16896\3303328325.py:1: SyntaxWarning: invalid escape sequence '\$'
  merged_df['Balance'] = merged_df['Balance'].replace('[\$,]', '', regex=True).astype(float)


### Cantidad de transacciones por tipo (Depósitos vs Retiros)

In [48]:
# Contar cuántas veces se hizo un depósito (> 0) y un retiro (> 0)
num_depositos = (merged_df['Deposits'] > 0).sum()
num_retiros = (merged_df['Withdrawls'] > 0).sum()

source = ColumnDataSource(data=dict(
    tipo=['Depósitos', 'Retiros'],
    cantidad=[num_depositos, num_retiros],
    color=['green', 'red']
))

p4 = figure(x_range=['Depósitos', 'Retiros'], 
            title="Número de Transacciones por Tipo",
            height=400, width=600, toolbar_location=None)

p4.vbar(x='tipo', top='cantidad', width=0.5, color='color', source=source)
p4.yaxis.axis_label = "Cantidad de transacciones"

show(p4)


### Top 10 clientes con mayor saldo final

In [49]:
clientes_saldo = merged_df.sort_values('Date').groupby('Customer Id').last().reset_index()

top_clientes = clientes_saldo.nlargest(10, 'Balance')[['Customer Id', 'First Name', 'Last Name', 'Balance']]

top_clientes['Nombre'] = top_clientes['First Name'] + ' ' + top_clientes['Last Name']

source = ColumnDataSource(data=dict(
    nombre=top_clientes['Nombre'],
    saldo=top_clientes['Balance']
))

p5 = figure(x_range=top_clientes['Nombre'], 
            title="Top 10 Clientes con Mayor Saldo Final",
            height=400, width=900, toolbar_location=None)

p5.vbar(x='nombre', top='saldo', width=0.6, source=source, color='navy')
p5.xaxis.major_label_orientation = 1.2
p5.yaxis.axis_label = 'Saldo Final (USD)'

show(p5)
